# Homework
The goal of this homework is to get familiar with tools like MLflow for experiment tracking and model management.

# Setup


## Import libraries

In [24]:
import requests
import os

from mlflow.tracking import MlflowClient

## Define paths

In [14]:
# Directory of the raw data files
_data_root = './data'

# Create the dataset directory
!mkdir -p {_data_root}

PATH_RAW_DATA_JAN_2022 = f'{_data_root}/green_tripdata_2022-01.parquet'
PATH_RAW_DATA_FEB_2022 = f'{_data_root}/green_tripdata_2022-02.parquet'
PATH_RAW_DATA_MAR_2022 = f'{_data_root}/green_tripdata_2022-03.parquet'

URL_RAW_DATA_JAN_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet'
URL_RAW_DATA_FEB_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet'
URL_RAW_DATA_MAR_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet'

TAXI_DATA_FOLDER = _data_root

## Download data

In [15]:
def download_file(url: str, save_path: str) -> None:
    """Download any type of file and save it locally.

    Args:
        url (str): URL of data file
        save_path (str): Local path to save the file
    """

    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

try:
    download_file(URL_RAW_DATA_JAN_2022, PATH_RAW_DATA_JAN_2022)
    download_file(URL_RAW_DATA_FEB_2022, PATH_RAW_DATA_FEB_2022)
    download_file(URL_RAW_DATA_MAR_2022, PATH_RAW_DATA_MAR_2022)
    print(f"Data downloaded, check the folder {_data_root}")
except Exception as e:
    print(f"Error: {e}")


Data downloaded, check the folder ./data


# Q1. Install the package
To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command `mlflow --version`  and check the output.

What's the version that you have?

In [1]:
# Checking version
!mlflow --version

mlflow, version 2.3.2


# Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from [here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

* load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
* fit a DictVectorizer on the training set (January 2022 data),
* save the preprocessed datasets and the DictVectorizer to disk.


Your task is to download the datasets and then execute this command:

```bash
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

In [16]:
! python3 preprocess_data.py --raw_data_path {TAXI_DATA_FOLDER} --dest_path ./output

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

So what's the size of the saved DictVectorizer file?

In [21]:
def get_file_size(file_path):
    size_in_bytes = os.path.getsize(file_path)
    return size_in_bytes / 1000

# Usage
file_path = './output/dv.pkl'
file_size_in_kb = get_file_size(file_path)
print(f"File size: {file_size_in_kb} KB")

File size: 153.66 KB


# Q3. Train a model with autolog
We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder homework.

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

**Tip 1**: don't forget to wrap the training code with a `with mlflow.start_run()`: statement as we showed in the videos.

**Tip 2**: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

**What is the value of the `max_depth` parameter:**

In [23]:
!python3 train.py

2023/06/01 12:25:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/miniconda3/envs/venv-mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [25]:
client = MlflowClient()
client.search_runs(experiment_ids='0')[0].data.params['max_depth']

'10'

# Launch the tracking server locally for MLflow
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

In case of MLflow, you need to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called artifacts for the artifacts store.


You should keep the tracking server running to work on the next three exercises that use the server.

In [28]:
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

[2023-06-01 12:45:38 -0600] [30270] [INFO] Starting gunicorn 20.1.0
[2023-06-01 12:45:38 -0600] [30270] [INFO] Listening at: http://127.0.0.1:5000 (30270)
[2023-06-01 12:45:38 -0600] [30270] [INFO] Using worker: sync
[2023-06-01 12:45:38 -0600] [30272] [INFO] Booting worker with pid: 30272
[2023-06-01 12:45:38 -0600] [30273] [INFO] Booting worker with pid: 30273
[2023-06-01 12:45:38 -0600] [30274] [INFO] Booting worker with pid: 30274
[2023-06-01 12:45:38 -0600] [30275] [INFO] Booting worker with pid: 30275
^C
[2023-06-01 12:45:43 -0600] [30270] [INFO] Handling signal: int
[2023-06-01 12:45:43 -0600] [30275] [INFO] Worker exiting (pid: 30275)
[2023-06-01 12:45:43 -0600] [30273] [INFO] Worker exiting (pid: 30273)
[2023-06-01 12:45:43 -0600] [30274] [INFO] Worker exiting (pid: 30274)
[2023-06-01 12:45:43 -0600] [30272] [INFO] Worker exiting (pid: 30272)


**Note**: This code was executed on a terminal. It is not possible to run the code in the background from a notebook.

# Q4. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `optuna`. We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the objective function during the optimization,
* the RMSE obtained on the validation set (February 2022 data).


**What's the best validation RMSE that you got?**